In [61]:
from IPython.display import clear_output

import pandas as pd
import numpy as np

import requests

frost_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'
frost_secert = '4fbc591a-214d-4066-a7a2-d268cf98f3eb'


In [139]:
def get_json(endpoint,params = {},clint_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    r = requests.get(endpoint, params, auth=(clint_id,''))
    json = r.json()
    if r.status_code == 200:
        data = json['data']
        print('Data retrieved from frost.met.no')
    else:
        print('Error! Returned status code %s' % r.status_code, '; Message: %s' % json['error']['message'], '; Reason: %s' % json['error']['reason'])
        return -1
    return(data)


def get_df(endpoint,params = {},client_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    '''from example code, needs to be tailored for each call (not good)'''
    data = get_json(endpoint,params,client_id)
    df = pd.DataFrame()
    for i in range(len(data)):
        row = pd.DataFrame(data[i]['observations'])
        row['referenceTime'] = data[i]['referenceTime']
        row['sourceId'] = data[i]['sourceId']
        df = df.append(row)

    return(df.reset_index())
    

def get_df(json):
    #not really tested enough
    df = pd.io.json.json_normalize(json)
    
    return(df)
    

In [35]:
location_json = get_json('https://frost.met.no/locations/v0.jsonld')
location_df = get_df(location_json)
location_df = location_df[~location_df['geometry.coordinates'].isnull()]

Data retrieved from frost.met.no


In [36]:
station_df = pd.read_csv('../csv/stations.csv')

In [58]:
def parse_coord(coord):
    split = coord.split(',')
    x = float(split[0][1:])
    y = float(split[1][1:-1])
    return(x,y)


def find_coord_distance(coord1,coord2):
    '''finds coordinate distance between coordinate on string form "[x,y]" '''
    if type(coord1) == str:
        x1,y1 = parse_coord(coord1)
    else:
        x1 = coord1[0]
        y1 = coord1[1]
    if type(coord2) == str:
        x2,y2 = parse_coord(coord2)
    else:
        x2 = coord2[0]
        y2 = coord2[1]
       
    return(np.sqrt((x2-x1)**2+(y2-y1)**2))
    
    
    

In [128]:
params = {
    'elements': 'air_temperature',
    'referencetime': '2018-01-01/2018-05-01'
}

In [148]:
params = {
    'elements': 'air_temperature,
    'referencetime': '2018-01-01/2018-05-01',
}

SyntaxError: EOL while scanning string literal (<ipython-input-148-5f3ead83b894>, line 2)

In [140]:
location_df['station_id'] = ''
for i,location in location_df.iloc[:10].iterrows():
    
    clear_output(wait=True)
    print(i, "/", len(location_df)) 
    
    dist_list = []
    station_list = []
    for j,station in station_df.iterrows():
        
        dist_list.append(find_coord_distance(location['geometry.coordinates'],station['geometry.coordinates']))
        station_list.append(station['id'])

    
    #test if we can retrieve data for 2018 for the closest station; if not, take the second closest etc.
    loc_json = -1
    while loc_json == -1:
        
        min_station = station_list[np.argmin(dist_list)]
        
        loc_params = params.copy()
        loc_params['sources'] = 'SN18390'
        
        loc_json = get_json('https://frost.met.no/observations/v0.jsonld',loc_params)
        
        np.delete(station_list,np.argmin(dist_list))
        np.delete(dist_list,np.argmin(dist_list))
        

    
    location_df.loc[i,'station_id'] = min_station
        

0 / 1342
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No d

KeyboardInterrupt: 

In [ ]:
location_df.drop(['geometry.@type','geometry.nearest'],axis=1,inplace=True)
location_df.to_csv('../csv/locations.csv')

In [151]:
loc_params

{'elements': 'air_temperature',
 'referencetime': '2018-01-01/2018-05-01',
 'sources': 'SN18390'}

In [160]:
get_json('https://frost.met.no/climatenormals/v0.jsonld?sources=SN18670')[0]

Data retrieved from frost.met.no


{'elementId': 'integral_of_deficit_interpolated(mean(air_temperature P1D) P1Y 17.0)',
 'normal': 4057.5,
 'period': '1961/1990',
 'sourceId': 'SN18670'}

In [ ]:

    for j,station in station_df.iterrows():
        
        dist_list.append(find_coord_distance(location['geometry.coordinates'],station['geometry.coordinates']))
        station_list.append(station['id'])
